## Modelos para Classificação Textual de Manifestações

> - Albertina PT-BR

### Imports e Configurações

Importação de dependências e funções utilitárias desenvolvidas para o auxílio nas atividades de tokenização, manipulação do conjunto de dados para a tarefa de classificação textual e conversão de tokens para as diferentes representações dos dados requeridas pelos modelos utilizads neste notebook.

Ambiente de Execução

In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-1ed8c666-34c8-7707-18f0-d13cac717d1f)


Conectando ao drive e definindo base_path

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
# SE FOR RODAR NO GOOGLE COLAB TEM QUE COLOCAR
# O CAMINHO COMPLETO PARA O NOTEBOOK NO BASEPATH
base_path = './'
%cd {base_path}
!pwd

Instalando dependências necessárias

In [ ]:
%%capture
!pip install bitsandbytes accelerate loralib
!pip install transformers[torch]
!pip install transformers[sentencepiece] # pro deberta baixar o tokenizer https://stackoverflow.com/questions/65431837/transformers-v4-x-convert-slow-tokenizer-to-fast-tokenizer
!pip install peft

Imports necessários

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from peft import prepare_model_for_kbit_training

from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig

import torch

from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from utils.plotting import print_confusion_matrix_as_table
from utils.dataframes import compute_tokenized_columns_in_dataframe, mapping_str_class_to_target

Configurações globais

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('classic')

### Parâmetros do Experimento

In [ ]:
EXP_ID = '/exp01'
EXP_DATASET_ID = '/01'
EXP_SEED = 42
EXP_SAMPLER_STRATEGY = None # 'not minority' | 'not majority' | None
EXP_SAMPLER_CLASS = None # RandomUnderSampler | RandomOverSampler | None
EXP_DESCRIPTION = 'Descrição do Experimento'

### Criando diretórios do experimento

In [ ]:
import os
path_to_results = '../results'
path_to_folder = path_to_results + EXP_ID
path_to_images = path_to_folder + '/images'
path_to_reports = path_to_folder + '/reports'
path_to_models = path_to_folder + '/models'
path_to_params = path_to_folder + '/params'
path_to_history = path_to_folder + '/history'
path_to_cmatrix = path_to_folder + '/cmatrix'

if os.path.isdir(path_to_folder) == False:
  os.mkdir(path_to_folder)
  os.mkdir(path_to_images)
  os.mkdir(path_to_reports)
  os.mkdir(path_to_models)
  os.mkdir(path_to_params)
  os.mkdir(path_to_history)
  os.mkdir(path_to_cmatrix)
  print('Diretório {} criado com sucesso!'.format(path_to_folder))
else:
  print('Diretório {} já existe!'.format(path_to_folder))

### Carregamento dos dados

Carregando dados das manifestações utilizadas no treinamento do classificador atual.

In [ ]:
# CONFIGUAR CAMINHO PARA CONJUNTOS DE TREINAMENTO, VALIDAÇÃO E TESTE
path_to_dataset = '../dataset' + EXP_DATASET_ID

train = pd.read_csv(path_to_dataset + '/train.csv', sep = ';')
test = pd.read_csv(path_to_dataset + '/test.csv', sep = ';')
valid = pd.read_csv(path_to_dataset + '/valid.csv', sep = ';')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
valid.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
valid.info()

In [ ]:
train['Assunto'].value_counts()

In [ ]:
test['Assunto'].value_counts()

In [ ]:
valid['Assunto'].value_counts()

### Tratamento dos dados

Identificando e removendo dados faltantes

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
valid.isna().sum()

In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)
valid.dropna(inplace=True)

In [ ]:
train['Texto'] = train['Texto'].astype('string')
train['Assunto'] = train['Assunto'].astype('string')
test['Texto'] = test['Texto'].astype('string')
test['Assunto'] = test['Assunto'].astype('string')
valid['Texto'] = valid['Texto'].astype('string')
valid['Assunto'] = valid['Assunto'].astype('string')

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
valid.info()

In [ ]:
!pip install unidecode
from unidecode import unidecode

print(sorted(train['Assunto'].unique()))
print(sorted(test['Assunto'].unique()))
print(sorted(valid['Assunto'].unique()))

def remove_accent(val):
    return unidecode(val)

train['Assunto'] = train['Assunto'].apply(remove_accent)
test['Assunto'] = test['Assunto'].apply(remove_accent)
valid['Assunto'] = valid['Assunto'].apply(remove_accent)

print(sorted(train['Assunto'].unique()))
print(sorted(test['Assunto'].unique()))
print(sorted(valid['Assunto'].unique()))

### Divisão do dados

In [ ]:
print('Total de exemplos no conjunto de treino:', len(train))
print('Total de exemplos no conjunto de teste :', len(test))
print('Total de exemplos no conjunto de valid :', len(valid))

### Pré-processamento

Aqui, adicionaremos novas colunas com o identificador do assunto de cada manifestação baseado no campo *Assunto*. Os assuntos serão categorizados em ordem alfabética e o indice de cada um na lista de categorias será utilizado para definição do identificador do assunto em valor numérico. Primeiro no conjunto de treinamento e, em seguida, no de teste.

In [ ]:
train = mapping_str_class_to_target(train)

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
test = mapping_str_class_to_target(test)

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
valid = mapping_str_class_to_target(valid)

In [ ]:
valid.head()

In [ ]:
valid.describe()

In [ ]:
target_names = sorted(train['Assunto'].astype('string').unique())
print(target_names)
target_names_masked = ['Classe {}'.format(i) for i in range(len(target_names))]
print(target_names_masked)

### Balanceamento

Identificamos que o conjunto de dados possui mais assuntos de determinadas classes do que de outras. Esta distribuição dos dados pode ser bastante prejudicial para a performance dos modelos e, portanto, foi necessário ajustar o conjunto de treinamento para que o total de exemplos de cada classe fique mais balanceado ao moldes do que é feito em https://medium.com/analytics-vidhya/re-sampling-imbalanced-training-corpus-for-sentiment-analysis-c9dc97f9eae1.

In [ ]:
original = train['Target'].value_counts()
index = np.arange(len(original))
fig, ax = plt.subplots()
width = 0.35
ax.bar(index, original, label = 'Original')
ax.set_xticks(index, original.index)
ax.legend()
plt.title('Divisão das classes antes do balanceamento')
#plt.show()
plt.savefig(path_to_images + '/balanceamento_antes.png')

In [ ]:
train_resampled = train

if EXP_SAMPLER_CLASS != None and EXP_SAMPLER_STRATEGY != None:
  sampler = EXP_SAMPLER_CLASS(sampling_strategy=EXP_SAMPLER_STRATEGY, random_state = EXP_SEED)
  train_resampled, _ = sampler.fit_resample(train_resampled, train_resampled['Target'])

In [ ]:
resample = train_resampled['Target'].value_counts()
index = np.arange(len(original))
fig, ax = plt.subplots()
width = 0.35
ax.bar(index + width/2, resample, label = 'Resample')
ax.set_xticks(index, original.index)
ax.legend()
plt.title('Divisão das classes após o balanceamento')
#plt.show()
plt.savefig(path_to_images + '/balanceamento_depois.png')

In [ ]:
train = train_resampled

### HUGGINGFACE

In [ ]:
model_huggingface_id = 'albertina'
model_huggingface_name = 'Albertina'
model_huggingface_url = 'PORTULAN/albertina-ptbr' #ESTAVA USANDO O BASE O TEMPO TODO!!!

#### Tokenização

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained(model_huggingface_url)

Como estamos realizando o finetuning de um modelo pré-treinado, temos que obedecer o formato de input dos dados usado no treinamento do modelo original, que, neste caso, foi de até 512 caracteres por sentença. O nosso dataset contém avaliações superiores a este valor e tivemos que truncar algumas dessas sentenças conforme é feito em
https://stackoverflow.com/questions/60551906/tensorflow-huggingface-invalid-argument-indices0-624-624-is-not-in-0.

In [ ]:
# def truncate_text_for_bert(data, max_size = 512):
#     truncated_text = []
#     for i, text in enumerate(data):
#         words = text.strip()
#         size = len(words)
#         out = words[0:min(size, max_size)]
#         truncated_text.append(out)
#     return truncated_text

# train_bert_text = truncate_text_for_bert(train['Texto'].to_list())
# valid_bert_text = truncate_text_for_bert(valid['Texto'].to_list())
# test_bert_text = truncate_text_for_bert(test['Texto'].to_list())

train_bert_text = train['Texto'].to_list()
valid_bert_text = valid['Texto'].to_list()
test_bert_text = test['Texto'].to_list()

Preparando conjunto de treinamento e teste a partir do tokenizador do bert

In [ ]:
# https://huggingface.co/docs/transformers/pad_truncation
X_train = bert_tokenizer(train_bert_text, return_tensors = "np", padding = True, truncation=True)
y_train = train['Target']
X_valid = bert_tokenizer(valid_bert_text, return_tensors = "np", padding = True, truncation=True)
y_valid = valid['Target']
X_test = bert_tokenizer(test_bert_text, return_tensors = "np", padding = True, truncation=True)
y_test = test['Target']

In [ ]:
X_train[0]

#### Modelo

In [ ]:
# isso só funciona com máquina com GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
# desabilidando 4bit por conta problema do save_pretrained:
# NotImplementedError: You are calling `save_pretrained` on
# a 4-bit converted model. This is currently not supported
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
# )
# bnb_config = None

bert_model = AutoModelForSequenceClassification.from_pretrained(
    model_huggingface_url,
    num_labels = 10,
    quantization_config=bnb_config,)

print(bert_model.get_memory_footprint())

In [ ]:
bert_model

#### Preparação

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_custom_dataset = CustomDataset(X_train, y_train)
valid_custom_dataset = CustomDataset(X_valid, y_valid)
test_custom_dataset = CustomDataset(X_test, y_test)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(bert_model)

In [ ]:
bert_model.gradient_checkpointing_enable()
bert_model = prepare_model_for_kbit_training(bert_model)

In [ ]:
print_trainable_parameters(bert_model)

In [ ]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    inference_mode = False, # TODO testar sem isso
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

bert_model = get_peft_model(bert_model, config)
print_trainable_parameters(bert_model)

#### Treinamento

In [ ]:
learning_rate = 5e-05
num_epochs = 4
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
batch_size = 8

In [ ]:
# https://discuss.huggingface.co/t/cuda-out-of-memory-error/17959

path_to_output = f"/tmp/{model_huggingface_id}"

training_args = TrainingArguments(
                                  output_dir=path_to_output,
                                  # You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported
                                  save_strategy = "epoch", # no
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  num_train_epochs = num_epochs,
                                  per_device_eval_batch_size=batch_size,
                                  per_device_train_batch_size=batch_size,
                                  run_name=model_huggingface_id,
                                  learning_rate = learning_rate,
                                  save_total_limit=2,
                                  load_best_model_at_end=True,
                                  logging_dir=f"{path_to_output}/logs",
                                  logging_strategy="steps",
                                  logging_steps=100,
                                  )

In [ ]:
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_custom_dataset,
    eval_dataset=valid_custom_dataset,
)

In [ ]:
trainer.train()

In [ ]:
path_to_log_history = path_to_history + f'/{model_huggingface_id}.csv'
log_history = pd.DataFrame(trainer.state.log_history)
log_history.to_csv(path_to_log_history, sep = ';', index = False)

#### Avaliação

In [ ]:
pred_output = trainer.predict(test_custom_dataset)

In [ ]:
predicted4 = np.argmax(pred_output.predictions, axis=1)

In [ ]:
print(metrics.classification_report(y_test, predicted4, target_names = target_names, zero_division = 0))

In [ ]:
cmatrix = confusion_matrix(y_test, predicted4)
display = ConfusionMatrixDisplay(
    confusion_matrix=cmatrix,
    display_labels=target_names_masked)

fig, ax = plt.subplots()
ax.set_title('Matriz de Confusão para o modelo {}\n'.format(model_huggingface_name))
display.plot(ax = ax, xticks_rotation = 'vertical')
fig.savefig(path_to_images + '/matriz-confusao-{}.png'.format(model_huggingface_id), bbox_inches='tight')

In [ ]:
np.save(path_to_cmatrix + '/{}'.format(model_huggingface_id), cmatrix)
np.load(path_to_cmatrix + '/{}.npy'.format(model_huggingface_id))

### Salvando modelos

In [ ]:
path_to_bert = path_to_models + f'/{model_huggingface_id}'
bert_tokenizer.save_pretrained(path_to_bert)
bert_model.save_pretrained(path_to_bert)
#loaded_tokenizer = AutoTokenizer.from_pretrained(path_to_bert)
#loaded_model = TFAutoModelForSequenceClassification.from_pretrained(path_to_bert)

### Preview da Execução

In [ ]:
MODEL_NAMES = [model_huggingface_name]
PREDICTS = [predicted4]
REPORTS = []

for index, model_name in enumerate(MODEL_NAMES):
    report = metrics.classification_report(y_test, PREDICTS[index], target_names = target_names, zero_division = 0, output_dict = True)
    REPORTS.append(report)

In [ ]:
from IPython.display import HTML, Markdown, Latex

Markdown(print_confusion_matrix_as_table(REPORTS, MODEL_NAMES))

### Salvando Resultados

In [ ]:
import json

for index, model_name in enumerate(MODEL_NAMES):
    filename = path_to_reports + '/' + model_name.lower() + '.json'
    with open(filename, 'w') as outfile:
        dictionary = REPORTS[index]
        json.dump(dictionary, outfile)

### Salvando Metadados

In [ ]:
metadados = dict({"experiment" : EXP_ID,
             "dataset" : EXP_DATASET_ID,
             "seed": EXP_SEED,
             "strategy" : str(EXP_SAMPLER_STRATEGY),
             "sampler": str(EXP_SAMPLER_CLASS),
             "description": EXP_DESCRIPTION})

path_to_metadata = path_to_params + '/metadata.json'

with open(path_to_metadata, 'w') as outfile:
  json.dump(metadados, outfile)

### Liberando memória

In [ ]:
import gc
gc.collect()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# import IPython
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True) # restart false (não funcionou)